In [ ]:

import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import matplotlib
import random
import pickle
import cv2
import os

# Parameters:-
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

# Input image:-
imagePaths = sorted(list(paths.list_images("/content/drive/MyDrive/Colab Notebooks/Projects/Traffic_Image_Classification/Dataset")))
random.seed(42)
random.shuffle(imagePaths)

# Create an list:-
data=[]
labels=[]

# loop over the input images
for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image,(IMAGE_DIMS[1],IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
  

	l = label = imagePath.split(os.path.sep)[-2].split('_')
 
	labels.append(l)

# Convert into numpy both input & lables:-
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# binarizer implementation
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)


#  loop over each of the possible class labels and show them
for (i, label) in enumerate(mlb.classes_):
	print("{}. {}".format(i + 1, label))

# Split Training & Testing
(trainX, testX, trainY, testY) = train_test_split(data,labels,test_size=0.2, random_state=42)

# # LENET ARCHITECTURE CODE

# initialize the model
model = Sequential()

# Rows & Columns  
imgRows=IMAGE_DIMS[0]
imgCols=IMAGE_DIMS[1]
numChannels=IMAGE_DIMS[2]
numClasses=3
inputShape = (imgRows, imgCols, numChannels)

activation="relu"
weightsPath=None

# define the first set of CONV => ACTIVATION => POOL layers
model.add(Conv2D(20, 5, padding="same",input_shape=inputShape))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# define the second set of CONV => ACTIVATION => POOL layers
model.add(Conv2D(50, 5, padding="same"))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# define the first FC => ACTIVATION layers
model.add(Flatten())
model.add(Dense(500))
model.add(Activation(activation))

# define the second FC layer
model.add(Dense(numClasses))


# lastly, define the soft-max classifier
model.add(Activation("softmax"))

# if a weights path is supplied (inicating that the model was
# pre-trained), then load the weights
if weightsPath is not None:
  model.load_weights(weightsPath)

#compile 
model.compile(loss = "CategoricalCrossentropy",optimizer = 'adam',metrics = ['accuracy'])

# # fitting the model 
hist = model.fit(x=trainX,y=trainY,epochs = 11,batch_size = 9,validation_data =(testX,testY),verbose = 1)

# evaluate the model
test_score = model.evaluate(testX,testY)
print("Test loss {:.5f},accuracy {:.3f}".format(test_score[0],test_score[1]*100))

# Save the model
# model.save('TRAINING_EXPERIENCE.h5')

# f = open("mlb.pickle", "wb")
# f.write(pickle.dumps(mlb))
# f.close()
# print(trainX.shape)
# print(testX.shape)

# print(trainY.shape)
# print(testY.shape)

1. critical
2. good
3. warning
Epoch 1/11
4/4 [==============================] - 2s 347ms/step - loss: 1.8772 - accuracy: 0.3871 - val_loss: 1.1837 - val_accuracy: 0.1250
Epoch 2/11
4/4 [==============================] - 1s 285ms/step - loss: 1.0905 - accuracy: 0.3548 - val_loss: 1.0314 - val_accuracy: 0.7500
Epoch 3/11
4/4 [==============================] - 1s 282ms/step - loss: 1.0094 - accuracy: 0.7419 - val_loss: 0.9390 - val_accuracy: 0.7500
Epoch 4/11
4/4 [==============================] - 1s 292ms/step - loss: 0.8301 - accuracy: 0.6129 - val_loss: 1.2400 - val_accuracy: 0.2500
Epoch 5/11
4/4 [==============================] - 1s 284ms/step - loss: 0.5453 - accuracy: 0.9355 - val_loss: 1.2756 - val_accuracy: 0.5000
Epoch 6/11
4/4 [==============================] - 1s 290ms/step - loss: 0.2705 - accuracy: 0.9355 - val_loss: 1.8602 - val_accuracy: 0.7500
Epoch 7/11
4/4 [==============================] - 1s 280ms/step - loss: 0.3223 - accuracy: 0.8387 - val_loss: 4.0539 - val_accura

In [11]:
# OWN DATA SET IMAGE CLASSIFICATION USING LENET-ARCHITECTURE MODEL:-

# Importing keras libraries and packages
from keras.models import load_model
from keras.preprocessing.image import img_to_array
import matplotlib
import numpy as np
import cv2
import pickle
import imutils

# Load Model:-
model = load_model("/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_image_classification/ARCHITECTURE/Models/TRAINING_EXPERIENCE.h5")  #important
mlb = pickle.loads(open("/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_image_classification/ARCHITECTURE/Models/mlb.pickle","rb").read())   #important

# Read an Input image:-
image = cv2.imread('/content/drive/MyDrive/Colab Notebooks/Projects/Traffic_Image_Classification/Dataset/good/2.jpg')
output = imutils.resize(image,width=400)
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
proba = model.predict(image)[0]
print(proba)
idxs = np.argsort(proba)[::-1][:2]
print(idxs)

for (i, j) in enumerate(idxs):
	label = "{}: {:.2f}%".format(mlb.classes_[j], proba[j] * 100)
	cv2.putText(output, label, (10, (i * 30) + 25), 
		cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
 
cv2.imwrite('/content/drive/MyDrive/Colab Notebooks/Projects/Projects/Traffic_image_classification/TASK/Sample_output/critical_2.jpg',output)


[2.1528844e-05 9.9991953e-01 5.8832367e-05]
[1 2]


True

# New Section